# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from contextlib import redirect_stdout
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'hardware-acquisition'
compute_threshold_method = 'top_n'  # top_n, window_percentile
compute_threshold = 10  # e.g. 10 to select top 10; 75 to select top 25%
variant = 'update-May-13'

estimation_method_lookup = {
    'cloud': estimate_cloud_costs,
    'amortized': estimate_amortized_hardware_costs,
    'hardware-acquisition': estimate_hardware_acquisition_cost,
    'hardware-capex-opex': estimate_hardware_capex_opex,
}
cost_estimation_function = estimation_method_lookup[estimation_method]

results_dir = f'results/{estimation_method}-{compute_threshold_method}={compute_threshold}-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_threshold_method=compute_threshold_method, compute_threshold=compute_threshold,
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(67, 5510, 142)

# Cost estimation

In [6]:
with open(f'{results_dir}/cost_estimation.out', 'w') as f:
    with redirect_stdout(f):
        cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

In [7]:
if estimation_method == 'hardware-capex-opex':
    frontier_pcd_df_copy = frontier_pcd_df.copy()
    with open(f'{results_dir}/component_cost_estimation.out', 'w') as f:
        with redirect_stdout(f):
            component_cost_df = cost_estimation_function(frontier_pcd_df_copy, hardware_df, price_df, separate_components=True)

In [8]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Organization categorization (from Organization),Possibly over 1e23 FLOP,Cost
82,Gemini Ultra,"Multimodal,Language,Vision","Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,633.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,Industry,NaN,1.257104e+09
98,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,Hosted access (no API),https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,checked,NaN,NaN,NaN,NaN,NaN,"via Pi, no API",Industry,NaN,2.558168e+08
123,Grok-1,Language,"Language modelling,Chat",NaN,SOTA improvement,"""On these benchmarks, Grok-1 displayed strong ...",Open source,"https://x.ai/model-card/, https://x.ai/blog/gr...",NaN,Announcing Grok,...,checked,2,7.0,NaN,Unreleased,Unreleased,apache 2.0,Industry,checked,NaN
138,ChatGLM3,"Multimodal,Language,Vision","Chat,Visual question answering",NaN,SOTA improvement,"Aiming at GPT-4V, ChatGLM3 has implemented ite...",NaN,https://www.zhipuai.cn/en/news/76,NaN,Zhipu AI launches third-generation base model,...,checked,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
184,Falcon-180B,Language,Language modelling,"Ebtesam Almazrouei, Hamza Alobeidli, Abdulaziz...",SOTA improvement,"""It's currently at the top of the Hugging Face...",Open access (restricted use),https://falconllm.tii.ae/falcon-180b.html; htt...,86.0,The Falcon Series of Open Language Models,...,NaN,NaN,NaN,17694720.0,NaN,NaN,"""Falcon 180b can be commercially used but unde...",Government,NaN,1.171712e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,11578.0,Xception: Deep Learning with Depthwise Separab...,...,NaN,NaN,NaN,43200.0,NaN,NaN,NaN,Industry,NaN,5.658000e+05
1159,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,Hosted access (no API),https://arxiv.org/abs/1609.08144,6196.0,Google's Neural Machine Translation System: Br...,...,NaN,NaN,NaN,414720.0,NaN,NaN,presumably deployed via Google translate,Industry,NaN,9.052800e+05
1201,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14887.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
1205,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,156882.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN


In [9]:
cost_df['Cost'].notna().sum()

40

Use the below to check data availability for specific systems

In [10]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [11]:
cost_df['Cost'].dropna()

82      1.257104e+09
98      2.558168e+08
184     1.171712e+08
223     2.860625e+07
235     2.196960e+07
319     7.151562e+08
335     5.858560e+07
390     1.098480e+07
405     1.521324e+08
430     3.295440e+07
431     2.196960e+07
453     1.638349e+08
496     2.929280e+07
506     1.404299e+08
528     2.074564e+07
557     2.340499e+07
561     8.298255e+07
602     1.281560e+08
609     2.929280e+07
651     1.296602e+06
659     1.037282e+07
697     2.074564e+07
713     2.074564e+07
716     2.031360e+07
785     1.983750e+08
820     5.078400e+06
825     2.074564e+07
852     7.779614e+06
856     1.037282e+07
870     1.015680e+07
871     1.015680e+07
896     2.031360e+07
982     5.186409e+06
1055    2.243042e+06
1105    4.715000e+05
1127    6.518016e+05
1147    6.029165e+04
1158    5.658000e+05
1159    9.052800e+05
1207    3.014582e+04
Name: Cost, dtype: float64

In [12]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [13]:
cost_df['Cost (inflation-adjusted)'].dropna()

82      1.257104e+09
98      2.564157e+08
184     1.178972e+08
223     2.876940e+07
235     2.209490e+07
319     7.228321e+08
335     5.933382e+07
390     1.120566e+07
405     1.551991e+08
430     3.361178e+07
431     2.240785e+07
453     1.669272e+08
496     2.975628e+07
506     1.427554e+08
528     2.144727e+07
557     2.415251e+07
561     8.563291e+07
602     1.322090e+08
609     3.027273e+07
651     1.340874e+06
659     1.072699e+07
697     2.143536e+07
713     2.152880e+07
716     2.108045e+07
785     2.082238e+08
820     5.325832e+06
825     2.179482e+07
852     8.290130e+06
856     1.105351e+07
870     1.083301e+07
871     1.083301e+07
896     2.186192e+07
982     5.586772e+06
1055    2.424971e+06
1105    5.092799e+05
1127    7.117793e+05
1147    6.583959e+04
1158    6.178640e+05
1159    9.885824e+05
1207    3.353512e+04
Name: Cost (inflation-adjusted), dtype: float64

In [14]:
assert cost_df['Cost (inflation-adjusted)'].notna().sum() == cost_df['Cost'].notna().sum()

# Regression

In [15]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [16]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     92.69
Date:                Mon, 13 May 2024   Prob (F-statistic):           9.69e-12
Time:                        18:20:11   Log-Likelihood:                -30.898
No. Observations:                  40   AIC:                             65.80
Df Residuals:                      38   BIC:                             69.17
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -734.4643     77.034     -9.534      0.000    -890.412    -578.517
x1             0.3670      0.038      9.628      0.000       0.290       0.444
==============================================================================
Omnibus:                        0.724   Durbin-Watson:                   1.883
Prob(Omnibus):                  0.696   Jarque-Bera (JB):                0.320
Skew:                          -0.216   Prob(JB):                        0.852
Kurtosis:                       3.078   Cond. No.                     1.83e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.83e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
with open(f'{results_dir}/regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(reg_results)
print_growth_rates(reg_results)

N=40.0
R^2=0.71
0.37 OOMs/year (95% CI: 0.29, 0.44)
2.3x/year (95% CI: 1.9x, 2.8x)
doubling time of 10 months (95% CI: 8, 12)


In [18]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [19]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,5.042748,0.239531,4.557843,5.527653,3.851489,6.234006,2015.00000,2015-01-01
1,5.079819,0.235935,4.602193,5.557444,3.891505,6.268133,2015.10101,2015-02-06
2,5.116889,0.232347,4.646527,5.587251,3.931476,6.302303,2015.20202,2015-03-15
3,5.153960,0.228768,4.690844,5.617076,3.971403,6.336517,2015.30303,2015-04-21
4,5.191031,0.225197,4.735143,5.646919,4.011286,6.370776,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,8.564475,0.165354,8.229733,8.899216,7.426048,9.702902,2024.59596,2024-08-06
96,8.601546,0.168668,8.260094,8.942997,7.461127,9.741964,2024.69697,2024-09-12
97,8.638616,0.172005,8.290410,8.986823,7.496158,9.781075,2024.79798,2024-10-19
98,8.675687,0.175363,8.320683,9.030692,7.531138,9.820236,2024.89899,2024-11-25


# Export data

In [20]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
82,Gemini Ultra,"Multimodal,Language,Vision","Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,1.257104e+09,1.257104e+09
98,Inflection-2,Language,Language modelling,Hosted access (no API),Inflection-2: The Next Step Up,2023-11-22,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,2.558168e+08,2.564157e+08
123,Grok-1,Language,"Language modelling,Chat",Open source,Announcing Grok,2023-11-04,xAI,3.140000e+11,2.900000e+24,NaN,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,ChatGLM3,"Multimodal,Language,Vision","Chat,Visual question answering",NaN,Zhipu AI launches third-generation base model,2023-10-27,Zhipu AI,1.300000e+11,1.092000e+24,1.050000e+12,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,Falcon-180B,Language,Language modelling,Open access (restricted use),The Falcon Series of Open Language Models,2023-09-06,Technology Innovation Institute,1.800000e+11,3.760000e+24,2.625000e+12,...,NVIDIA A100 SXM4 40 GB,United Arab Emirates,NaN,NaN,4096.0,0.1876,Amazon Web Services,NaN,1.171712e+08,1.178972e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,Xception,Vision,Image classification,NaN,Xception: Deep Learning with Depthwise Separab...,2016-10-07,Google,2.285595e+07,4.360000e+20,3.500000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,60.0,NaN,NaN,NaN,5.658000e+05,6.178640e+05
1159,GNMT,Language,Translation,Hosted access (no API),Google's Neural Machine Translation System: Br...,2016-09-26,Google,2.780000e+08,6.900000e+21,3.600000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,9.052800e+05,9.885824e+05
1201,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27,DeepMind,NaN,1.900000e+21,2.940000e+07,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1205,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10,Microsoft,6.000000e+07,1.210000e+19,1.280000e+06,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)

# Plots

In [22]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [23]:
label_systems = ['GNMT', 'AlphaGo Master', 'AlphaGo Zero', 'AlphaZero', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra', 'Inflection-2']

tpu_mask = cost_df['Training hardware'].str.contains('TPU', na=False)
tpu_cost_df = cost_df.loc[tpu_mask]
gpu_cost_df = cost_df.loc[~tpu_mask]

fig = px.scatter(
    gpu_cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    log_y=True,
)
fig.add_scatter(
    x=tpu_cost_df['Publication date'],
    y=tpu_cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    marker_symbol='circle-open' if estimation_method != 'cloud' else 'circle',
    name='Using equivalent price of TPU' if estimation_method != 'cloud' else '',
    showlegend=estimation_method != 'cloud',
)
fig.add_scatter(
    x=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)
fig.add_scatter(
    x=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])
if estimation_method == 'hardware-acquisition':
    fig.update_yaxes(range=[4, 10])
else:
    fig.update_yaxes(range=[1, 9])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Cost components

In [24]:
cost_component_names = [
    'AI accelerator chip cost',
    'Other server components cost',
    'Cluster-level interconnect cost',
    'Energy cost',
]

In [25]:
for key in cost_component_names:
    component_cost_df[f"{key} (%)"] = component_cost_df[key] / component_cost_df['Cost'] * 100
component_cost_df['AI accelerator chip cost (%)']

NameError: name 'component_cost_df' is not defined

In [ ]:
# Stacked bar chart of cost components, using component_cost_df
cost_component_pc_names = [name + ' (%)' for name in cost_component_names]
fig = px.bar(
    component_cost_df.dropna(subset=cost_component_pc_names).sort_values(by='Publication date'),
    x='System',
    y=cost_component_pc_names,
    barmode='stack',
)

# axis labels
fig.update_xaxes(title_text='ML model')
fig.update_yaxes(title_text='% of hardware CapEx + OpEx')
fig.update_layout(
    legend=dict(
        title_text='Cost component',
        x=0.60,
        y=0.05,
    )
)
# limits 0 to 100
fig.update_yaxes(range=[0, 100])

fig.update_yaxes(tickvals=list(range(0, 101, 10)))

# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_component_percentage')

fig.show()

In [26]:
# Stacked bar chart of cost components, using component_cost_df
fig = px.bar(
    component_cost_df.dropna(subset=['Cost']).sort_values(by='Publication date'),
    x='System',
    y='Energy cost (%)',
    barmode='stack',
    # labels='Cost %',
    # text='Energy cost %',
)
# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Energy cost (% of Hardware CapEx + OpEx)')
# fig.update_layout(
#     legend=dict(
#         title_text='Cost component',
#         x=0.75,
#         y=0.05,
#     )
# )
# limits 0 to 100
fig.update_yaxes(range=[0, 30])
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_percentage')

fig.show()

NameError: name 'component_cost_df' is not defined

In [ ]:
# Stacked bar chart of cost components, using component_cost_df
component_cost_df = component_cost_df.dropna(subset=['Cost'])
component_cost_df.loc[:, 'Hardware cost'] = [cost['hardware'] for cost in component_cost_df['Cost']]
component_cost_df.loc[:, 'Energy cost'] = [cost['energy'] for cost in component_cost_df['Cost']]
fig = px.scatter(
    component_cost_df.dropna(subset=['Cost']).sort_values(by='Publication date'),
    x='Publication date',
    y='Energy cost',
    text='System',
)
# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Energy cost')
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_cost')

fig.show()

In [ ]:
from energy import energy_price

# Stacked bar chart of cost components, using component_cost_df
component_cost_df = component_cost_df.dropna(subset=['Cost'])
component_cost_df.loc[:, 'Hardware cost'] = [cost['hardware'] for cost in component_cost_df['Cost']]
component_cost_df.loc[:, 'Energy (kWh)'] = [row['Cost']['energy'] / energy_price(row['Publication date'].year) for _, row in component_cost_df.iterrows()]
fig = px.scatter(
    component_cost_df,
    x='Publication date',
    y='Energy (kWh)',
    text='System',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_kwh')

fig.show()

In [ ]:
component_cost_df.loc[:, 'Power capacity for 3-month training run (kW)'] = [row['Cost']['energy'] / energy_price(row['Publication date'].year) / (HOURS_PER_YEAR/4) / server_TDP_fraction(row['Training hardware']) for _, row in component_cost_df.iterrows()]
fig = px.scatter(
    component_cost_df,
    x='Publication date',
    y='Power capacity for 3-month training run (kW)',
    text='System',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'power_capacity_kw')

fig.show()

In [ ]:
component_cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(component_cost_df['Publication date']))

In [ ]:
energy_reg_results = fit_ols_regression(component_cost_df, ['Publication date (float)'], 'Power capacity for 3-month training run (kW)', logy=True)
energy_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     87.41
Date:                Thu, 09 May 2024   Prob (F-statistic):           4.10e-12
Time:                        12:27:03   Log-Likelihood:                -41.068
No. Observations:                  47   AIC:                             86.14
Df Residuals:                      45   BIC:                             89.84
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -725.0174     77.807     -9.318      0.000    -881.728    -568.307
x1             0.3600      0.039      9.349      0.000       0.282       0.438
==============================================================================
Omnibus:                        3.175   Durbin-Watson:                   2.213
Prob(Omnibus):                  0.204   Jarque-Bera (JB):                2.964
Skew:                           0.553   Prob(JB):                        0.227
Kurtosis:                       2.461   Cond. No.                     1.82e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""